# Linear regression 

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline  

# Prepare train data
train_X = np.linspace(-1, 1, 200)
train_Y = 2 * train_X + np.random.randn(*train_X.shape) * 0.33 + 10
plt.plot(train_X,train_Y,"+")
plt.show()


In [ ]:
# The optimal goal is to learn [W, b] which is [2, 10]
# Define the model
X = tf.placeholder("float")
Y = tf.placeholder("float")
w = tf.Variable(0.0, name="weight")
b = tf.Variable(0.0, name="bias")
loss = tf.square(Y - X*w - b) # mean square loss

In [ ]:
#Define the optmization operation.
# doing w = w - \alpha * dw  (gradient descent)

train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)


In [ ]:
# Create session to run
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    epoch = 1
    for i in range(40):
        for (x, y) in zip(train_X, train_Y):
            _, w_value, b_value = sess.run([train_op, w, b],feed_dict={X: x,Y: y})
        print("Epoch: {}, w: {}, b: {}".format(epoch, w_value, b_value))
        epoch += 1


#draw
plt.plot(train_X,train_Y,"+")
plt.plot(train_X,train_X.dot(w_value)+b_value)
plt.show()

# Convolutional Neural Network 

refs: https://github.com/nlintz/TensorFlow-Tutorials/blob/master/05_convolutional_net.ipynb



![CNN architecture](http://adventuresinmachinelearning.com/wp-content/uploads/2017/04/Typical_cnn.png)

[image source](http://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img


In [ ]:
# Import `matplotlib`
import matplotlib.pyplot as plt
import os

%matplotlib inline 
# chose some images to display
choosen_ind = [300, 2250, 3650, 4000]

# Fill out the subplots with the random images and add shape, min and max values
for i in range(len(choosen_ind)):
    plt.subplot(1, 4, i+1)
    plt.axis('off')
    plt.imshow(trX[i,:,:,0])
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    print('label: ', np.argmax(trY[i]))

In [ ]:
batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, w_o):
    with tf.variable_scope('fist_conv_max'):
        l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                            strides=[1, 1, 1, 1], padding='SAME'))
        l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                            strides=[1, 2, 2, 1], padding='SAME')
        #l1 = tf.nn.dropout(l1, p_keep_conv)
    with tf.variable_scope('second_conv_max'):
        l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                            strides=[1, 1, 1, 1], padding='SAME'))
        l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                            strides=[1, 2, 2, 1], padding='SAME')
        #l2 = tf.nn.dropout(l2, p_keep_conv)
    with tf.variable_scope('third_conv_max'):
        l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                            strides=[1, 1, 1, 1], padding='SAME'))
        l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                            strides=[1, 2, 2, 1], padding='SAME')
    with tf.variable_scope('reshape_fully_connect'):
        l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
        #l3 = tf.nn.dropout(l3, p_keep_conv)

        l4 = tf.nn.relu(tf.matmul(l3, w4))
        #l4 = tf.nn.dropout(l4, p_keep_hidden)

        pyx = tf.matmul(l4, w_o)
    return pyx



In [ ]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

#p_keep_conv = tf.placeholder("float")
#p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o)


In [ ]:
# define the output and optimizer

cost = tf.reduce_mean( tf.square(py_x - Y))
train_op = tf.train.GradientDescentOptimizer(1).minimize(cost)
predict_op = tf.argmax(py_x, 1)
tf.summary.scalar('loss', cost)
merged_summary = tf.summary.merge_all()

In [ ]:
log_dir = os.path.join('.', 'log_dir_mnist')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

steps = 0

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()
    
    file_writer = tf.summary.FileWriter(log_dir, sess.graph)
    
    for i in range(100):
        train_ind = np.arange(len(trX))
        np.random.shuffle(train_ind)
        print(train_ind[0:10])
        trX = trX[train_ind]
        trY = trY[train_ind]
        
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            this_summ, this_cost, _ = sess.run([merged_summary,cost,  train_op], 
                                               feed_dict={X: trX[start:end], Y: trY[start:end]})
            
            file_writer.add_summary(this_summ, steps)
            steps += 1
            
            if steps%10==0:
                print('steps: ', steps, 'loss is: ', this_cost)
        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices]}
                                                         )))

In [ ]:
# go through https://github.com/aymericdamien/TensorFlow-Examples


# Sentiment Classification

# IMDB DATASET

>Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

>As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.



In [1]:
'''Train a recurrent convolutional network on the IMDB sentiment
classification task.
'''
import os
import numpy as np

import tensorflow as tf

from keras.layers import Input, Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D

import keras.backend as K
from keras.preprocessing import sequence
from keras.datasets import imdb
max_features = 10000
maxlen = 100  # the max length of each setence.

print('Loading data...')
(x_train_org, y_train_org), (x_test_org, y_test_org) = imdb.load_data(num_words=max_features)

print((x_train_org.shape), 'train sequences')
print((x_test_org.shape), 'test sequences')

print('Pad sequences (samples x time)')

x_train = sequence.pad_sequences(x_train_org, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test_org, maxlen=maxlen)
y_train = y_train_org
y_test  = y_test_org
print('x_train shape:', x_train.shape)
print('x_test shape:',  x_test.shape)

Using TensorFlow backend.


Loading data...
(25000,) train sequences
(25000,) test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


# Each line is one sentence.
Each number represent one word. The the label contains only one value indicating the positive and negtive.


In [2]:
print('Negtive sentence\n')
print('the input sequence is:\n', x_train[1000])
print('the corresponding label is: ', y_train[1000])

print('\nPositive sentence\n')
print('the input sequence is:\n', x_train[-1])
print('the corresponding label is: ', y_train[-1])


Negtive sentence

the input sequence is:
 [   4 1654 1986    7    2   23   27   96    8    6 1304   19 4205    5    4
  500   10   10  323 3199 2102   16 1815 3009   34  167  670    2   37   12
  272   40   16  115   23    4  270    4   38  446  229    9   43   14  499
    7 4767   67   85  857   18    4  111    2  469    4  513    7    2   47
    6  171   52  388   21    9 1116 2262   34   78  802    4  277    2  131
  225    4    2    7    6   52  206 1154  133    6 2579 1991  310   19   53
 8904  206    5 4291 1062  238   60   30  184   52]
the corresponding label is:  0

Positive sentence

the input sequence is:
 [  78 1099   17 2345    2   21   27 9685 6139    5    2 1603   92 1183    4
 1310    7    4  204   42   97   90   35  221  109   29  127   27  118    8
   97   12  157   21 6789    2    9    6   66   78 1099    4  631 1191    5
 2642  272  191 1070    6 7585    8 2197    2    2  544    5  383 1271  848
 1468    2  497    2    8 1597 8778    2   21   60   27  239    9   43 

# Word Embedding.
Now, each sentence only contains a list of Integer. But during learning, we need 
## There are two types of word encoding systems
1. One-hot encoding, each work is present as [1,0,0,0],... [0,0,0,1]. For the genetic cases, there are ATCG, only for characters. This type of coding usually works just fine.

2. For this setiment task, the number of vocabulary can be of size 20,000. So instead of getting a vector of length that long. We can embedding them into a much more compact size, like 128. The embedding matrix in this case, is a $20000 \times 128$ matrix. And this matrix is a trainable parameter as well. For more information, please refer to [Word embedding](http://sebastianruder.com/word-embeddings-1/)


In [3]:
# Embedding

embedding_size = 128

#inputs = Input(batch_shape=(None, maxlen,))
inputs = tf.placeholder(tf.int64, shape = [None, maxlen])
out = Embedding(max_features, embedding_size, input_length=maxlen)(inputs)
out = Dropout(0.25)(out)
print("The embedding of the input sentence is of shape: ", out.get_shape())

The embedding of the input sentence is of shape:  (?, 100, 128)


# Convolution and maxpooling of the input sequence
1. 1d convolution can be used to extract features, it can be seen as fused the neighbooring words together to a more abstract representation.
2. Max pooling is applied to shrink the sentence length. In the following case, the input sentence is shrinked by 4 times.

In [4]:
# Convolution
kernel_size = 5
filters = 64
shrink_size = 4

out = Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1)(out)

out = MaxPooling1D(shrink_size)(out)

print('The size of the preprocessed sentences is: ', out.get_shape())


The size of the preprocessed sentences is:  (?, 24, 64)


# The RNN cell

<!---  ![RNN](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg)<element height="50%" width="50%"> -->

## The computation in a rnn cell
RNN compute update it's current hidden state based on current inputs(optional) and the hidden state from last time step. The connection that links it's current state and past state is called recurrent connections. 

### First of all, you need a initia state. 
$h_0 = \mathbf{0}$ 

### For $t$ from 1 to $T$:
$h_t = x_tW + h_{t-1}U + b$



In [5]:
def rnn_cell(rnn_input, state, input_size, state_size):      
    try:
        with tf.variable_scope('rnn_cell'):
            W = tf.get_variable('W', [input_size, state_size])
            U = tf.get_variable('U', [state_size, state_size])
            b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))
    except:
        pass

    with tf.variable_scope('rnn_cell', reuse=True):
        W = tf.get_variable('W', [input_size, state_size])
        U = tf.get_variable('U', [state_size, state_size])
        b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))
    
    return tf.tanh(tf.matmul(rnn_input, W) + tf.matmul(state, U)  + b)
 
def RNN(inputs, init_state, input_size,state_size, time_step):

    state = init_state
    for idx in range(time_step):
        with tf.name_scope('rnn_step_{}'.format(idx)):
            state = rnn_cell(inputs[:,idx,:], state, input_size, state_size)
    return state


state_size = 128  # the size of the hidden state.
rnn_input_size = filters 
time_steps = out.get_shape()[1]

init_state = tf.placeholder(tf.float32, [None, state_size])

out = RNN(out, init_state, rnn_input_size, state_size, time_steps)

print('Finished building the recurrent model.')

Finished building the recurrent model.


# Take the output of the RNN
1. Map it to a probability indicating the positive or negtive class.

In [6]:
out = Dense(1)(out)
out = tf.nn.sigmoid(out)

target = tf.placeholder(tf.float32, out.get_shape())

#losses and train_step
element_loss = K.binary_crossentropy(out, target, from_logits=False)
#element_loss = tf.square(out- target)

total_loss = tf.reduce_mean(element_loss)
tf.summary.scalar('loss', total_loss)
merged_summary = tf.summary.merge_all()

learning_rate = 1e-1
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

print('finish constructing the graph.')

finish constructing the graph.


In [7]:
saver = tf.train.Saver()
#sess = tf.InteractiveSession()
ckpt_dir = os.path.join('.', 'model')

def save_model(sess, saver, checkpoint_dir, model_name, step):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver.save(sess, os.path.join(checkpoint_dir, model_name), global_step=step)
    
def load_model(sess, saver, checkpoint_dir):
    print("[*] Reading checkpoints...")
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
        return True
    return False


# Train the Network.
1. Open a terminal, type the following to monitor the training procedure at http://127.0.1.1:6006/.
   ```
   tensorboard --logdir='./log_dir'
   ```
   

In [8]:
"""
Train the network
"""
batch_size = 30  # the batch_size used to train the network.
num_epochs = 100 # total number of epoches used to trian the network.


def Indexflow(Totalnum, batch_size, random=True):
    numberofchunk = int(Totalnum + batch_size - 1)// int(batch_size)   # the floor
    #Chunkfile = np.zeros((batch_size, row*col*channel))
    totalIndx = np.arange(Totalnum)
    if random is True:
        totalIndx = np.random.permutation(totalIndx)
    
    chunkstart = 0
    for chunkidx in range(int(numberofchunk)):
        thisnum = min(batch_size, Totalnum - chunkidx*batch_size)
        thisInd = totalIndx[chunkstart: chunkstart + thisnum]
        chunkstart += thisnum
        yield thisInd


log_dir = os.path.join('.', 'log_dir')

def train_network(num_epochs,  verbose=True):
    with tf.Session() as sess:
        print(sess.graph)
        file_writer = tf.summary.FileWriter(log_dir, sess.graph)

        sess.run(tf.global_variables_initializer())
        try:
            load_model(sess, saver, ckpt_dir)
        except:
            print('Failed to load saved model, start from random initialization.')
            
        training_losses = []
        totalnum = x_train.shape[0]
        step  = 0
        summary_freq = 50
        save_freq = 500
        
        for epoch in range(num_epochs):
            training_loss = 0
            for thisInd in Indexflow(totalnum, batch_size):
                x_batch = x_train[thisInd].astype(np.float32)
                y_batch = y_train[thisInd][:,None].astype(np.float32)
                init_state_ = np.zeros((x_batch.shape[0], state_size), dtype=np.float32)
                step += 1
                summary, losses, _  = sess.run([merged_summary, total_loss,train_step],
                                                 feed_dict={inputs: x_batch, 
                                                            target: y_batch,
                                                            init_state: init_state_,
                                                            K.learning_phase(): 1}
                                              )
                training_loss += losses
                file_writer.add_summary(summary, step)
                
                if step % summary_freq == 0 and step > 0:
                    if verbose:
                        print("Average loss at step", step,
                              "for last 100 steps:", training_loss/summary_freq)
                    training_losses.append(training_loss/summary_freq)
                    training_loss = 0
                if step % save_freq == 0 and step > 0:
                    save_model(sess, saver, ckpt_dir, 'model.name', step)
    return training_losses

train_network(100)

[*] Reading checkpoints...
Average loss at step 50 for last 100 steps: 1.05286389112
Average loss at step 100 for last 100 steps: 0.93197920084
Average loss at step 150 for last 100 steps: 0.850394327641
Average loss at step 200 for last 100 steps: 0.786729800701
Average loss at step 250 for last 100 steps: 0.98144269228
Average loss at step 300 for last 100 steps: 0.813057712317
Average loss at step 350 for last 100 steps: 0.759086581469
Average loss at step 400 for last 100 steps: 0.833176500797
Average loss at step 450 for last 100 steps: 0.810520368814
Average loss at step 500 for last 100 steps: 0.873464934826
Average loss at step 550 for last 100 steps: 1.08853302121
Average loss at step 600 for last 100 steps: 0.938685026169
Average loss at step 650 for last 100 steps: 0.799125505686
Average loss at step 700 for last 100 steps: 0.853672667742
Average loss at step 750 for last 100 steps: 0.795592676401
Average loss at step 800 for last 100 steps: 0.80948040247
Average loss at ste

Average loss at step 6750 for last 100 steps: 0.789415348768
Average loss at step 6800 for last 100 steps: 0.933137887716
Average loss at step 6850 for last 100 steps: 0.745569602251
Average loss at step 6900 for last 100 steps: 0.826289693117
Average loss at step 6950 for last 100 steps: 0.886658940315
Average loss at step 7000 for last 100 steps: 0.753519817591
Average loss at step 7050 for last 100 steps: 0.77233704567
Average loss at step 7100 for last 100 steps: 0.827213151455
Average loss at step 7150 for last 100 steps: 0.82847802639
Average loss at step 7200 for last 100 steps: 0.792974154949
Average loss at step 7250 for last 100 steps: 0.82409064889
Average loss at step 7300 for last 100 steps: 0.858510450125
Average loss at step 7350 for last 100 steps: 0.837886815071
Average loss at step 7400 for last 100 steps: 0.808259410858
Average loss at step 7450 for last 100 steps: 0.801323120594
Average loss at step 7500 for last 100 steps: 0.800808663368
Average loss at step 7550 f

Average loss at step 13450 for last 100 steps: 0.779603840113
Average loss at step 13500 for last 100 steps: 0.814087747335
Average loss at step 13550 for last 100 steps: 0.929437366724
Average loss at step 13600 for last 100 steps: 0.939226088524
Average loss at step 13650 for last 100 steps: 0.757536046505
Average loss at step 13700 for last 100 steps: 0.829314798117
Average loss at step 13750 for last 100 steps: 0.848580356836
Average loss at step 13800 for last 100 steps: 0.842262045145
Average loss at step 13850 for last 100 steps: 1.04379331231
Average loss at step 13900 for last 100 steps: 1.08498283982
Average loss at step 13950 for last 100 steps: 0.754808934927
Average loss at step 14000 for last 100 steps: 0.880482811928
Average loss at step 14050 for last 100 steps: 0.971935219765
Average loss at step 14100 for last 100 steps: 0.762451258898
Average loss at step 14150 for last 100 steps: 0.768637167215
Average loss at step 14200 for last 100 steps: 0.355932972431
Average lo

Average loss at step 20100 for last 100 steps: 0.785332745314
Average loss at step 20150 for last 100 steps: 0.867849030495
Average loss at step 20200 for last 100 steps: 0.776429787874
Average loss at step 20250 for last 100 steps: 0.828925091028
Average loss at step 20300 for last 100 steps: 0.798301700354
Average loss at step 20350 for last 100 steps: 0.836534019709
Average loss at step 20400 for last 100 steps: 0.786626189947
Average loss at step 20450 for last 100 steps: 0.825457669497
Average loss at step 20500 for last 100 steps: 0.907950203419
Average loss at step 20550 for last 100 steps: 0.769873428345
Average loss at step 20600 for last 100 steps: 0.859459750652
Average loss at step 20650 for last 100 steps: 0.747369452715
Average loss at step 20700 for last 100 steps: 0.758803489208
Average loss at step 20750 for last 100 steps: 0.78657122016
Average loss at step 20800 for last 100 steps: 1.07036514401
Average loss at step 20850 for last 100 steps: 1.01792040825
Average los

Average loss at step 26750 for last 100 steps: 0.776011167765
Average loss at step 26800 for last 100 steps: 0.904799002409
Average loss at step 26850 for last 100 steps: 0.960778089762
Average loss at step 26900 for last 100 steps: 0.781282446384
Average loss at step 26950 for last 100 steps: 0.764607845545
Average loss at step 27000 for last 100 steps: 0.845168157816
Average loss at step 27050 for last 100 steps: 0.866204053164
Average loss at step 27100 for last 100 steps: 0.743490610123
Average loss at step 27150 for last 100 steps: 0.839406981468
Average loss at step 27200 for last 100 steps: 0.789254211187
Average loss at step 27250 for last 100 steps: 0.840509073734
Average loss at step 27300 for last 100 steps: 0.748020591736
Average loss at step 27350 for last 100 steps: 0.867585482597
Average loss at step 27400 for last 100 steps: 0.806713947058
Average loss at step 27450 for last 100 steps: 0.748852669001
Average loss at step 27500 for last 100 steps: 0.764812122583
Average 

...

Average loss at step 60000 for last 100 steps: 0.91456474185
Average loss at step 60050 for last 100 steps: 0.0276050102711
Average loss at step 60100 for last 100 steps: 0.764664051533
Average loss at step 60150 for last 100 steps: 0.856839647293
Average loss at step 60200 for last 100 steps: 0.751307467222
Average loss at step 60250 for last 100 steps: 0.772349522114
Average loss at step 60300 for last 100 steps: 0.800594222546
Average loss at step 60350 for last 100 steps: 0.841287105083
Average loss at step 60400 for last 100 steps: 0.847219592333
Average loss at step 60450 for last 100 steps: 0.808486229181
Average loss at step 60500 for last 100 steps: 0.781887404919
Average loss at step 60550 for last 100 steps: 0.938682091236
Average loss at step 60600 for last 100 steps: 0.787795460224
Average loss at step 60650 for last 100 steps: 0.790819694996
Average loss at step 60700 for last 100 steps: 0.800205214024
Average loss at step 60750 for last 100 steps: 0.748110390902
Average 

Average loss at step 66650 for last 100 steps: 0.775708465576
Average loss at step 66700 for last 100 steps: 0.827561095953
Average loss at step 66750 for last 100 steps: 0.506326293945
Average loss at step 66800 for last 100 steps: 0.81321395874
Average loss at step 66850 for last 100 steps: 0.787022610903
Average loss at step 66900 for last 100 steps: 0.898806549311
Average loss at step 66950 for last 100 steps: 0.971841423512
Average loss at step 67000 for last 100 steps: 0.835799255371
Average loss at step 67050 for last 100 steps: 0.833322433233
Average loss at step 67100 for last 100 steps: 1.15050042033
Average loss at step 67150 for last 100 steps: 0.78180711627
Average loss at step 67200 for last 100 steps: 0.77956599474
Average loss at step 67250 for last 100 steps: 0.938865323067
Average loss at step 67300 for last 100 steps: 0.794261894226
Average loss at step 67350 for last 100 steps: 0.791354432106
Average loss at step 67400 for last 100 steps: 0.906051640511
Average loss

Average loss at step 73300 for last 100 steps: 0.7405527246
Average loss at step 73350 for last 100 steps: 0.760439784527
Average loss at step 73400 for last 100 steps: 0.130562577248
Average loss at step 73450 for last 100 steps: 0.865636115074
Average loss at step 73500 for last 100 steps: 0.854081935883
Average loss at step 73550 for last 100 steps: 0.863090543747
Average loss at step 73600 for last 100 steps: 0.886587232351
Average loss at step 73650 for last 100 steps: 0.78847899437
Average loss at step 73700 for last 100 steps: 0.75795609355
Average loss at step 73750 for last 100 steps: 0.773727880716
Average loss at step 73800 for last 100 steps: 0.797227808237
Average loss at step 73850 for last 100 steps: 1.2346457839
Average loss at step 73900 for last 100 steps: 0.874172064066
Average loss at step 73950 for last 100 steps: 0.765183807611
Average loss at step 74000 for last 100 steps: 0.825729734898
Average loss at step 74050 for last 100 steps: 0.864796862602
Average loss a

Average loss at step 79950 for last 100 steps: 0.938932425976
Average loss at step 80000 for last 100 steps: 0.853807878494
Average loss at step 80050 for last 100 steps: 0.806108102798
Average loss at step 80100 for last 100 steps: 0.568194856644
Average loss at step 80150 for last 100 steps: 0.824947046041
Average loss at step 80200 for last 100 steps: 0.789974269867
Average loss at step 80250 for last 100 steps: 1.1792767477
Average loss at step 80300 for last 100 steps: 0.840507316589
Average loss at step 80350 for last 100 steps: 0.922220025063
Average loss at step 80400 for last 100 steps: 0.94896073103
Average loss at step 80450 for last 100 steps: 0.818103284836
Average loss at step 80500 for last 100 steps: 0.812515683174
Average loss at step 80550 for last 100 steps: 0.875473091602
Average loss at step 80600 for last 100 steps: 0.858656263351
Average loss at step 80650 for last 100 steps: 0.971578508615
Average loss at step 80700 for last 100 steps: 0.909898412228
Average los

[1.0528638911247254,
 0.9319792008399963,
 0.85039432764053347,
 0.7867298007011414,
 0.98144269227981562,
 0.81305771231651303,
 0.7590865814685821,
 0.83317650079727168,
 0.81052036881446843,
 0.87346493482589727,
 1.0885330212116242,
 0.93868502616882321,
 0.79912550568580631,
 0.85367266774177553,
 0.79559267640113829,
 0.80948040246963504,
 0.24285692930221559,
 0.71911592721939088,
 1.0659229230880738,
 0.74668430447578427,
 0.84169348716735837,
 0.77762767195701599,
 0.76642739892005918,
 0.88802905797958376,
 0.8003810238838196,
 0.81373012185096738,
 0.80655039310455323,
 0.7485801291465759,
 0.80606162786483759,
 0.87291187405586246,
 0.84932038664817811,
 1.0687380349636078,
 1.0871946299076081,
 0.61766077876091008,
 0.77863313317298888,
 0.84047901630401611,
 0.81970775365829462,
 0.76926356792449946,
 0.88603796362876897,
 1.0499413776397706,
 0.87285285115242006,
 0.83016181826591495,
 0.84894394159317021,
 0.85149337172508244,
 0.80369734406471249,
 0.76987560153007506,

# Testing a trained model.
1. This time, we only need output of the graph.
2. Set the K.learning_phase to 0.

In [ ]:

testing_number = x_test.shape[0]
results_list = []

with tf.Session() as sess:
    load_model(sess, saver, ckpt_dir)
    for thisInd in Indexflow(testing_number, batch_size, False): 
        #Have to set random to False, otherwise, order is not maintaied
        test_batch = x_test[thisInd].astype(np.float32)
        init_state_ = np.zeros((test_batch.shape[0], state_size), dtype=np.float32)
        output  = sess.run(out,
                           feed_dict={inputs: test_batch, 
                                      init_state: init_state_,
                                      K.learning_phase(): 0}  
                             )
        
        #output is of size (batch_size , 1)
        results_list.append(output)
    results = np.concatenate(results_list, 0)
print('Finished predicting the testing batch.\n')

# The accurary on the testing data.

In [ ]:
predict = (results[:,0] > 0.5).astype(np.int)
diff = predict == np.array(y_test)
correct = np.sum(diff)
acc = correct/predict.shape[0]
print(acc)